# import vectors

In [ ]:
import pandas as pd
import pickle
from scipy import sparse

df = pd.read_csv("../00_data_cleaning/out/20newsgroup_preprocessed.csv")
labels_df = df["target"]

# Load TF-IDF vectors from NPZ file
with open("../01_vectorization/out/tf_idf/X_tfidf.npz", "rb") as f:
    X_tfidf = pickle.load(f)


# print("\nTF-IDF matrix shape:", X_tfidf.shape)

0    10
1     3
2    17
3     3
4     4
Name: target, dtype: int64


# import Doc2Vec

In [ ]:
# TODO: import doc2vec & sbert vectors

train/validate/test split on vectors & scaling

In [ ]:
from sklearn.model_selection import train_test_split


# train 0.7, val 0.15, test 0.15
X_train, X_val_test, y_train, y_val_test = train_test_split(
    X_tfidf, labels_df, test_size=0.7, random_state=42, shuffle=True, stratify=labels_df
)

X_val, X_test, y_val, y_test = train_test_split(
    X_val_test,
    y_val_test,
    test_size=0.5,
    random_state=42,
    shuffle=True,
    stratify=y_val_test,
)

# SVM

In [ ]:
import svm

# MLPClassifier

# Decision Tree